In [1]:
import os
import sys
import csv
from file_utils import *
import re

In [2]:
list_files = []
iemocap_full_release_path = "/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/"

iemocap_pre_processed_data_path = "/home/mandeep_stanford/cs231n_project/code/pre-processed_data/"

for x in range(5):
    sess_name = "Session" + str(x+1)
    print(sess_name)
    path = iemocap_full_release_path + sess_name + "/sentences/wav/"
    print(path)
    #file_search(path, list_files)
    #list_files = sorted(list_files)

    # print sess_name + ", #sum files: " + str(len(list_files))

Session1
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session1/sentences/wav/
Session2
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session2/sentences/wav/
Session3
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session3/sentences/wav/
Session4
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session4/sentences/wav/
Session5
/home/mandeep_stanford/cs231n_project/IEMOCAP_full_release/Session5/sentences/wav/


In [3]:
useful_regex = re.compile(r'\[.+\]\n', re.IGNORECASE)


In [4]:
file_path = iemocap_full_release_path + '/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt'

In [5]:
with open(file_path) as f:
    file_content = f.read()
    
info_lines = re.findall(useful_regex, file_content)

In [6]:
for l in info_lines[1:10]:
    print(l.strip().split('\t'))

['[6.2901 - 8.2357]', 'Ses01F_impro01_F000', 'neu', '[2.5000, 2.5000, 2.5000]']
['[10.0100 - 11.3925]', 'Ses01F_impro01_F001', 'neu', '[2.5000, 2.5000, 2.5000]']
['[14.8872 - 18.0175]', 'Ses01F_impro01_F002', 'neu', '[2.5000, 2.5000, 2.5000]']
['[19.2900 - 20.7875]', 'Ses01F_impro01_F003', 'xxx', '[2.5000, 3.0000, 3.0000]']
['[21.3257 - 24.7400]', 'Ses01F_impro01_F004', 'xxx', '[2.5000, 3.0000, 2.5000]']
['[27.4600 - 31.4900]', 'Ses01F_impro01_F005', 'neu', '[2.5000, 3.5000, 2.0000]']
['[38.9650 - 43.5900]', 'Ses01F_impro01_F006', 'fru', '[2.0000, 3.5000, 3.5000]']
['[46.5800 - 52.1900]', 'Ses01F_impro01_F007', 'fru', '[2.5000, 3.5000, 3.5000]']
['[56.1600 - 58.8225]', 'Ses01F_impro01_F008', 'fru', '[2.0000, 3.5000, 3.5000]']


In [7]:
info_line = re.compile(r'\[.+\]\n', re.IGNORECASE)

start_times, end_times, wav_file_names, emotions, vals, acts, doms = [], [], [], [], [], [], []
import pdb
for sess in range(1, 5):
    emo_evaluation_dir = iemocap_full_release_path + '/Session{}/dialog/EmoEvaluation/'.format(sess)
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        #
        #
        x = re.search("^Ses.*", file)
        #print(x, file)
        #pdb.set_trace()
        if x == None:
            continue
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        info_lines = re.findall(info_line, content)
        for line in info_lines[1:]:  # the first line is a header
            #pdb.set_trace()   
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            start_time, end_time = start_end_time[1:-1].split('-')
            val, act, dom = val_act_dom[1:-1].split(',')
            val, act, dom = float(val), float(act), float(dom)
            start_time, end_time = float(start_time), float(end_time)
            start_times.append(start_time)
            end_times.append(end_time)
            wav_file_names.append(wav_file_name)
            emotions.append(emotion)
            vals.append(val)
            acts.append(act)
            doms.append(dom)

In [8]:
import pandas as pd

df_iemocap = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion', 'val', 'act', 'dom'])

df_iemocap['start_time'] = start_times
df_iemocap['end_time'] = end_times
df_iemocap['wav_file'] = wav_file_names
df_iemocap['emotion'] = emotions
df_iemocap['val'] = vals
df_iemocap['act'] = acts
df_iemocap['dom'] = doms

df_iemocap.tail()

,start_time,end_time,wav_file,emotion,val,act,dom
7864,139.08,141.80,Ses04F_impro01_M021,ang,2.0,4.0,4.5
7865,145.26,148.57,Ses04F_impro01_M022,fru,1.5,4.0,4.5
7866,148.69,151.21,Ses04F_impro01_M023,ang,1.5,4.5,5.0
7867,151.22,155.47,Ses04F_impro01_M024,ang,1.0,4.5,5.0
7868,165.43,168.51,Ses04F_impro01_M025,xxx,2.0,3.0,3.5


In [9]:
df_iemocap.to_csv(iemocap_pre_processed_data_path + '/df_iemocap.csv', index=False)

In [10]:
print("Extracting Audio Vectors now")

Extracting Audio Vectors now


In [11]:
#import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install pysoundfile

#you may have to execute this in unix terminal to instal sndfile.h
#sudo apt-get install libsndfile1

In [12]:
import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import pickle

import IPython.display
import librosa.display
ms.use('seaborn-muted')
%matplotlib inline

In [13]:
file_path = iemocap_full_release_path + "/Session1/dialog/wav/Ses01F_impro01.wav"

y, sr = librosa.load(file_path, sr=44100)
y, sr

(array([ 0.42572615,  0.48587543,  0.37312022, ..., -0.31514615,
        -0.16263676,  0.        ], dtype=float32), 44100)

In [14]:
import pandas as pd
import math

labels_df = pd.read_csv(iemocap_pre_processed_data_path + "/df_iemocap.csv")
iemocap_dir = iemocap_full_release_path

In [15]:
print("loop thru wav audio files and dump audio vector into pkl format")

loop thru wav audio files and dump audio vector into pkl format


In [16]:
labels_df

,start_time,end_time,wav_file,emotion,val,act,dom
0,5.3913,8.2350,Ses01M_script01_2_F000,ang,2.5,2.5,3.0
1,12.7400,17.2800,Ses01M_script01_2_F001,ang,2.5,2.0,3.0
2,32.6300,35.3425,Ses01M_script01_2_F002,xxx,2.5,2.5,3.0
3,36.7600,39.4800,Ses01M_script01_2_F003,xxx,2.5,3.0,4.0
4,41.1800,44.7000,Ses01M_script01_2_F004,ang,1.5,4.0,4.0
...,...,...,...,...,...,...,...
7864,139.0800,141.8000,Ses04F_impro01_M021,ang,2.0,4.0,4.5
7865,145.2600,148.5700,Ses04F_impro01_M022,fru,1.5,4.0,4.5
7866,148.6900,151.2100,Ses04F_impro01_M023,ang,1.5,4.5,5.0
7867,151.2200,155.4700,Ses04F_impro01_M024,ang,1.0,4.5,5.0


In [ ]:
sr = 44100
audio_vectors = {}
import pdb
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

for sess in [1]:  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = '{}/Session{}/dialog/wav/'.format(iemocap_dir, sess)
    #pdb.set_trace()
    orig_wav_files = os.listdir(wav_file_path)
    for orig_wav_file in tqdm(orig_wav_files):
        #print("Wav file is ", orig_wav_file)
        x = re.search("^Ses.*", orig_wav_file)
        if x == None:
            #print("Skiping file", orig_wav_file)
            continue
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            orig_wav_file, file_format = orig_wav_file.split('.')
            print("Working on file - ", orig_wav_file)
            #pdb.set_trace()
            for index, row in labels_df[labels_df['wav_file'].str.contains(orig_wav_file)].iterrows():
                #pdb.set_trace()
                #print("Inside for loop")
                #orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
                #orig_wav_file, file_format = orig_wav_file.split('.')
                start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                
                
                X = librosa.stft(truncated_wav_vector)
                Xdb = librosa.amplitude_to_db(abs(X))
                plt.figure(figsize=(14, 5))
                #plt.figure()
                librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
                plt.colorbar()
                
                pdb.set_trace()
                plt.savefig(iemocap_pre_processed_data_path + "/images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
                
                ###############
                #window_size = 1024
                #window = np.hanning(window_size)
                #stft  = librosa.core.spectrum.stft(truncated_wav_vector, n_fft=window_size, hop_length=512, window=window)
                #out = 2 * np.abs(stft) / np.sum(window)
                #fig = plt.Figure()
                #canvas = FigureCanvas(fig)
                #ax = fig.add_subplot(111)
                #p = librosa.display.specshow(librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis='log', x_axis='time')
                #fig.savefig('spec11.png')

                ###############
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector
                
        except:
            print('An exception occured for {}'.format(orig_wav_file))
    with open(iemocap_pre_processed_data_path + '/audio_vectors_exp_{}.pkl'.format(sess), 'wb') as f:
        pickle.dump(audio_vectors, f)

  0%|          | 0/56 [00:00<?, ?it/s]

Working on file -  Ses01F_script03_1
> <ipython-input-17-97e5983de447>(27)<module>()
-> start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
(Pdb) c
> <ipython-input-17-97e5983de447>(41)<module>()
-> plt.savefig(iemocap_pre_processed_data_path + "/images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
(Pdb) c
> <ipython-input-17-97e5983de447>(23)<module>()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-17-97e5983de447>(41)<module>()
-> plt.savefig(iemocap_pre_processed_data_path + "/images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
(Pdb) c
> <ipython-input-17-97e5983de447>(23)<module>()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-17-97e5983de447>(41)<module>()
-> plt.savefig(iemocap_pre_processed_data_path + "/images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
(Pdb) c
> <ipython-input-17-